In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

在GCP上进行端到端的机器学习：MLOps阶段6：开始使用Vertex AI批量预测自定义表格模型

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_custom_tabular_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/ml_ops/stage6/get_started_with_custom_tabular_model_batch.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/ml_ops/stage6/get_started_with_custom_tabular_model_batch.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>        
</table>
<br/><br/><br/>

## 概述

本教程演示了如何从自定义表格模型进行批量预测。

###目标

在本教程中，您将学习如何使用自定义表格模型进行“Vertex AI批量预测”。

本教程使用以下Google Cloud ML服务和资源：

- “Vertex AI批量预测”
- “Vertex AI模型”

执行的步骤包括：

- 将预训练的表格模型上传为“Vertex AI模型”资源。
- 对“模型”资源进行批量预测，以JSONL格式。
- 对“模型”资源进行批量预测，以CSV格式。
- 对“模型”资源进行批量预测，以BigQuery格式。

使用批量预测和使用在线预测之间有一个关键差异：

* 预测服务：对整个实例集（即一个或多个数据项）进行按需预测，并即时返回结果。

* 批量预测服务：对后台中的整个实例集进行排队（批量）预测，并在准备就绪时将结果存储在Cloud Storage存储桶中。

### 模型

**JSONL 批量输入示例**

本教程中的示例使用了一个预训练模型，其中模型构件存储在一个公共云存储桶中。

该模型基于[博客文章](https://cloud.google.com/blog/topics/developers-practitioners/churn-prediction-game-developers-using-google-analytics-4-ga4-and-bigquery-ml)。该模型的理念是，您的公司拥有详细的日志数据，描述了游戏用户与网站的互动方式。原始数据包含以下信息类别：

- 身份 - 独特的玩家身份编号
- 人口特征 - 有关玩家的信息，如玩家所在的地理区域
- 行为特征 - 玩家触发某些游戏事件的次数，比如达到新的级别
- 流失倾向 - 这是标签或目标特征，它提供了关于玩家可能流失的估计概率，即停止成为活跃玩家。

**CSV 批量输入示例**

本教程中的示例使用了一个预训练模型，其中模型构件存储在一个公共云存储桶中。该模型是在[TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/iris)中的[Iris数据集](https://www.tensorflow.org/datasets/catalog/overview)上训练的XGBoost模型。训练模型预测三种类型的鸢尾花的类别：山鸢尾、维吉尼亚鸢尾或变色鸢尾。

### 费用

本教程使用了 Google Cloud 的计费组件：

* Vertex AI
* BigQuery
* Cloud Storage

了解 [Vertex AI 的价格](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 的价格](https://cloud.google.com/storage/pricing)，并使用 [价格计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用情况生成一个费用估算。

## 安装

安装以下包以执行这个笔记本。

In [ ]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-aiplatform
! pip3 install {USER_FLAG} --quiet --upgrade google-cloud-bigquery

### 重新启动内核

在安装了额外的软件包之后，您需要重新启动笔记本内核，以便它可以找到这些软件包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## 在开始之前

### GPU运行时

*如果有这个选项，请确保在GPU运行时中运行此笔记本。在Colab中，选择* **运行时 > 更改运行时类型 > GPU**

### 设置您的Google Cloud项目

**无论您使用的是哪种笔记本环境，以下步骤都是必需的。**

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300免费信用额度，可用于计算/存储成本。

2. [确保为您的项目启用计费。](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [启用以下API：Vertex AI APIs，Compute Engine APIs和Cloud Storage。](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component,storage-component.googleapis.com)

4. 如果您在本地运行此笔记本，您需要安装[Cloud SDK]((https://cloud.google.com/sdk))。

5. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在本笔记本中的所有命令中使用正确的项目。

**注意**：Jupyter会将以`！`为前缀的行解析为shell命令，并会解析以`$`为前缀的Python变量。

#### 设置您的项目ID

**如果您不知道您的项目ID**，您可以通过使用`gcloud`来获取您的项目ID。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = ! gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

地区

您还可以更改`REGION`变量，该变量用于本笔记本其余部分的操作。以下是Vertex AI支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太：`asia-east1`

您可能无法使用多区域存储桶进行Vertex AI的训练。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

UUID

如果您正在进行在线教程会话，您可能正在使用共享的测试账户或项目。为了避免在创建的资源上发生名称冲突，您可以为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google Cloud帐户

**如果您正在使用Vertex AI Workbench笔记本**，您的环境已经经过身份验证。

**如果您正在使用Colab**，运行下面的单元格，并按照提示进行身份验证OAuth。

**否则**，请按照以下步骤操作：

在Cloud控制台中，转到[创建服务帐户密钥](https://console.cloud.google.com/apis/credentials/serviceaccountkey)页面。

**单击创建服务帐户**。

在**服务帐户名称**字段中输入名称，然后单击**创建**。

在**将此服务帐户授权给项目**部分，单击角色下拉列表。在筛选框中键入“Vertex”，然后选择**Vertex管理员**。在筛选框中键入“Storage Object Admin”，然后选择**存储对象管理员**。

单击创建。包含您的密钥的JSON文件将下载到本地环境中。

在下面的单元格中将您的服务帐户密钥的路径输入为GOOGLE_APPLICATION_CREDENTIALS变量，然后运行该单元格。

In [ ]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Vertex AI Workbench, then don't execute this code
IS_COLAB = "google.colab" in sys.modules
if not os.path.exists("/opt/deeplearning/metadata/env_version") and not os.getenv(
    "DL_ANACONDA_HOME"
):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，都需要以下步骤。**

当您初始化用于Python的Vertex AI SDK时，您需要指定一个云存储分期桶。这个分期桶是您的数据集和模型资源在不同会话中保留的地方。

在下面设置您的云存储桶名称。桶名称必须在所有谷歌云项目中全局唯一，包括您组织之外的项目。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + UUID
    BUCKET_URI = "gs://" + BUCKET_NAME

只有当您的存储桶不存在时，请运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION $BUCKET_URI

最后，通过检查Cloud Storage桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

### 设定变量

接下来，设定一些在教程中使用的变量。
### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aiplatform
from google.cloud import bigquery

### 初始化用于 Python 的 Vertex AI SDK

为您的项目和相应的存储桶初始化用于 Python 的 Vertex AI SDK。

In [ ]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

### 创建BigQuery客户端

在本教程中，您将使用与训练预训练模型相同的公共BigQuery表中的数据。您将创建一个客户端接口，随后将用于访问数据。

In [ ]:
bqclient = bigquery.Client(project=PROJECT_ID)

#### 设置硬件加速器

您可以为训练和预测设置硬件加速器。

将变量`DEPLOY_GPU/DEPLOY_NGPU`设置为使用支持GPU的容器镜像和分配给虚拟机实例的GPU数量。例如，要使用一个带有4个 Nvidia Telsa K80 GPU 的容器镜像分配给每个虚拟机实例，您可以指定：

    (aip.AcceleratorType.NVIDIA_TESLA_K80, 4)

否则，请指定`(None, None)`来使用一个在CPU上运行的容器镜像。

了解更多关于[您所在地区的硬件加速器支持](https://cloud.google.com/vertex-ai/docs/general/locations#accelerators)。

*注意*：在 2.3 版本之前的 TF 发布中，对于 GPU 支持无法在此教程中加载自定义模型。这是一个已知问题，在 TF 2.3 中已经修复。这是由于在服务函数中生成的静态图操作导致的。如果您在自己的自定义模型上遇到此问题，请使用支持 GPU 的 TF 2.3 版本的容器镜像。

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_GPU"):
    DEPLOY_GPU, DEPLOY_NGPU = (
        aiplatform.gapic.AcceleratorType.NVIDIA_TESLA_K80,
        int(os.getenv("IS_TESTING_DEPLOY_GPU")),
    )
else:
    DEPLOY_GPU, DEPLOY_NGPU = (None, None)

将预先构建的容器设置为用于预测的 Docker 容器映像。 

有关最新列表，请查看[用于预测的预构建容器](https://cloud.google.com/ai-platform-unified/docs/predictions/pre-built-containers)。

In [ ]:
if DEPLOY_GPU:
    DEPLOY_VERSION = "tf2-gpu.2-5"
else:
    DEPLOY_VERSION = "tf2-cpu.2-5"

DEPLOY_IMAGE = "{}-docker.pkg.dev/vertex-ai/prediction/{}:latest".format(
    REGION.split("-")[0], DEPLOY_VERSION
)

print("Deployment:", DEPLOY_IMAGE, DEPLOY_GPU, DEPLOY_NGPU)

#### 设置机器类型

接下来，设置用于预测的机器类型。

- 将变量`DEPLOY_COMPUTE`设置为配置用于预测的虚拟机的计算资源。
 - `机器类型`
     - `n1-standard`：每个 vCPU 3.75GB 内存。
     - `n1-highmem`：每个 vCPU 6.5GB 内存
     - `n1-highcpu`：每个 vCPU 0.9 GB 内存
 - `vCPUs`：数量在\[2, 4, 8, 16, 32, 64, 96\]

*注意：您还可以使用n2和e2机器类型进行训练和部署，但它们不支持GPU。*

In [ ]:
if os.getenv("IS_TESTING_DEPLOY_MACHINE"):
    MACHINE_TYPE = os.getenv("IS_TESTING_DEPLOY_MACHINE")
else:
    MACHINE_TYPE = "n1-standard"

VCPU = "4"
DEPLOY_COMPUTE = MACHINE_TYPE + "-" + VCPU
print("Train machine type", DEPLOY_COMPUTE)

### 将模型工件上传为 `Vertex AI Model` 资源

首先，使用 `upload()` 方法将预训练的自定义表格模型工件作为 `Vertex AI Model` 资源上传，参数如下：

- `display_name`：`Model` 资源的可读名称。
- `artifact_uri`：模型工件的 Cloud Storage 位置。
- `serving_container_image`：将模型部署到 `Vertex AI Endpoint` 资源时要使用的服务容器映像。
- `sync`：是否等待过程完成，或立即返回（异步）。

In [ ]:
MODEL_ARTIFACT_URI = "gs://mco-mm/churn"

model = aiplatform.Model.upload(
    display_name="churn_" + UUID,
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True,
)

print(model)

批处理预测介绍

批处理预测提供了对大量预测请求进行离线批处理的能力。资源仅在批处理过程中分配，并在批处理请求完成后取消分配。结果存储在云存储中，与在线预测不同，在线预测的结果会作为HTTP响应包返回。

批处理作业的输入格式取决于您的模型服务器支持的格式。首先，您的模型服务器中的Web服务器必须支持JSONL格式，这将被Web服务器转换为模型输入接口或服务功能接口直接支持的格式。对于批处理预测，这种JSONL格式被称为“枢轴”格式。

批处理预测作业的输入格式

批处理服务器接受以下自定义表格模型的输入格式：

- JSONL
- CSV
- BigQuery表格

批处理预测作业的输出格式

批处理服务器接受以下自定义表格模型的输出格式：

- JSONL
- BigQuery表格（输入为BigQuery表格时）

枢轴格式

批处理服务器将输入格式转换为“枢轴”（JSONL）格式，如下所示：

JSONL

每个输入行（请求）应包含一个且仅一个有效的json值。

{"values": [1, 2, 3, 4], "key": 1}
{"values": [5, 6, 7, 8], "key": 2}

批处理服务器使用相同的格式生成枢轴数据。然后将生成的枢轴数据包装成一个负载请求：

{"instances": [
  {"values": [1, 2, 3, 4], "key": 1},
  {"values": [5, 6, 7, 8], "key": 2}
]}

CSV

第一行中的csv标题始终被忽略。字符串字段需要明确用双引号括起来，否则该行将被丢弃，并且会输出解析错误消息到错误文件。非引号值总是转换为浮点数。

col1,col2,col3
1,3,"cat1"
2,4,"cat2"

批处理服务器将每个输入行（请求）转换为一个JSON数组。

{"instances": [
 [1.0,3.0,"cat1"],
 [2.0,4.0,"cat2"]
]}

BigQuery

将每行转换为JSON数组。例如：

[1.0,3.0,"cat1"]
[2.0,4.0,"cat2"]

批处理服务器以相同的格式生成枢轴数据。然后将生成的枢轴数据包装成一个负载请求：

{"instances": [
 [1.0,3.0,"cat1"],
 [2.0,4.0,"cat2"]
]}

TFRecords

TFRecord文件中的实例由apache_beam.io.tfrecordio模块以二进制形式读取。然后将二进制对象序列化为ASCII字符串。预测服务器负责知道恢复实例的解码器。

{"instances": [
 {"b64","b64EncodedASCIIString"},
 {"b64","b64EncodedASCIIString"}
]}

FileList

FileList格式包含文件列表。在“FileList”文件中，每行指定一个文件路径，以云存储位置形式表示。

gs://my-bucket/file1.txt
gs://my-bucket/file2.txt

批处理服务器以二进制形式读取文件。然后将二进制对象序列化为ASCII字符串。

{"instances": [
 {"b64","b64EncodedASCIIString"},
 {"b64","b64EncodedASCIIString"}
]}

发出一个批量预测请求给你的`Vertex AI模型`资源。

生成批量预测数据

接下来，您从BigQuery训练表中提取前10个实例，用于批量预测请求。

In [ ]:
DATASET_BQ_URI = "bq://mco-mm.bqmlga4.train"
TARGET = "churned"

# Download the table.
table = bigquery.TableReference.from_string(DATASET_BQ_URI[5:])

rows = bqclient.list_rows(table, max_results=10)

instances = []
for row in rows:
    instance = {}
    for key, value in row.items():
        if key == TARGET:
            continue
        if value is None:
            value = ""
        instance[key] = value
    instances.append(instance)

print(instances[0])

准备 JSONL 格式的批量预测数据

在可以通过批量预测运行数据之前，您需要将数据保存成几种可能的格式之一。

在本教程中，请使用 JSONL 格式，因为它与每个图像当前表示的三维列表兼容。要做到这一点：

1. 在一个文件中，将每个实例单独写成 JSON。
2. 将此文件上传到 Cloud Storage。

有关批量预测输入格式的更多详细信息，请参阅：https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions#batch_request_input

In [ ]:
import json

BATCH_PREDICTION_INSTANCES_FILE = "batch_prediction_instances.jsonl"

BATCH_PREDICTION_GCS_SOURCE = (
    BUCKET_URI + "/batch_prediction_instances/" + BATCH_PREDICTION_INSTANCES_FILE
)

# Write instances at JSONL
with open(BATCH_PREDICTION_INSTANCES_FILE, "w") as f:
    for instance in instances:
        f.write(json.dumps(instance) + "\n")

# Upload to Cloud Storage bucket
! gsutil cp $BATCH_PREDICTION_INSTANCES_FILE $BATCH_PREDICTION_GCS_SOURCE

print("Uploaded instances to: ", BATCH_PREDICTION_GCS_SOURCE)

发送预测请求

要进行批量预测请求，请使用以下参数调用模型对象的`batch_predict`方法：
- `instances_format`：批量预测请求文件的格式："jsonl"、"csv"、"bigquery"、"tf-record"、"tf-record-gzip"或"file-list"
- `prediction_format`：批量预测响应文件的格式："jsonl"、"csv"、"bigquery"、"tf-record"、"tf-record-gzip"或"file-list"
- `job_display_name`：预测作业的易读名称。
- `gcs_source`：您的批量预测请求的一个或多个Cloud Storage路径的列表。
- `gcs_destination_prefix`：服务写入预测结果的Cloud Storage路径。
- `model_parameters`：用于提供预测结果的额外过滤参数。
- `machine_type`：用于训练的机器类型。
- `accelerator_type`：硬件加速器类型。
- `accelerator_count`：要附加到工作程序副本的加速器数量。
- `starting_replica_count`：初始提供计算实例的数量。
- `max_replica_count`：要扩展到的最大计算实例数量。在本教程中，仅提供一个实例。

### 计算实例扩展

您可以指定单个实例（或节点）来处理您的批量预测请求。本教程使用单个节点，因此变量`MIN_NODES`和`MAX_NODES`均设置为`1`。

如果您想要使用多个节点来处理您的批量预测请求，请将`MAX_NODES`设置为您想要使用的最大节点数量。Vertex AI会自动扩展用于提供您的预测的节点数量，最多扩展到您设置的最大数量。请参考[定价页面](https://cloud.google.com/vertex-ai/pricing#prediction-prices)了解使用多个节点进行自动扩展的成本。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

# The name of the job
BATCH_PREDICTION_JOB_NAME = "churn_batch-" + UUID

# Folder in the bucket to write results to
DESTINATION_FOLDER = "batch_prediction_results"

# The Cloud Storage bucket to upload results to
BATCH_PREDICTION_GCS_DEST_PREFIX = BUCKET_URI + "/" + DESTINATION_FOLDER

# Make SDK batch_predict method call
batch_prediction_job = model.batch_predict(
    instances_format="jsonl",
    predictions_format="jsonl",
    job_display_name=BATCH_PREDICTION_JOB_NAME,
    gcs_source=BATCH_PREDICTION_GCS_SOURCE,
    gcs_destination_prefix=BATCH_PREDICTION_GCS_DEST_PREFIX,
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=True,
)

### 获取预测结果

接下来，从完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的云存储输出桶中。您可以调用方法iter_outputs()来获取生成结果的每个云存储文件的列表。每个文件包含一个或多个以JSON格式表示的预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_prediction_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

#### 删除批量预测作业

您可以使用`delete()`方法来删除您的批量预测作业。

In [ ]:
batch_prediction_job.delete()

删除模型

您可以使用`delete()`方法删除模型。

In [ ]:
model.delete()

使用CSV作为输入格式进行批量预测

要使用CSV作为输入格式，您的模型必须将其输入作为值列表（数组）。先前的预训练模型（`churn`）将其输入作为字典（键/值对）。

在这个例子中，您上传了一个在鸢尾花数据集上训练的预训练XGBoost模型。这个模型将其输入作为一列值。

In [ ]:
MODEL_ARTIFACT_URI = "gs://cloud-samples-data/vertex-ai/google-cloud-aiplatform-ci-artifacts/models/iris_xgboost/"

DEPLOY_IMAGE = (
    f"{REGION.split('-')[0]}-docker.pkg.dev/vertex-ai/prediction/xgboost-cpu.1-5:latest"
)

model = aiplatform.Model.upload(
    display_name="churn_" + UUID,
    artifact_uri=MODEL_ARTIFACT_URI,
    serving_container_image_uri=DEPLOY_IMAGE,
    sync=True,
)

print(model)

### 为CSV准备批量预测请求

接下来，您需要将预测请求实例格式化为CSV格式。在本例中，您将从存储在公共云存储桶中的数据集中获取实例的数据。

In [ ]:
data_file = "gs://cloud-samples-data/ai-platform/iris/iris_data.csv"
! gsutil cat $data_file | head -n 10 > test.csv

! cat test.csv

BATCH_PREDICTION_INSTANCES_FILE = "batch_prediction_instances.csv"

BATCH_PREDICTION_GCS_SOURCE = (
    BUCKET_URI + "/batch_prediction_instances/" + BATCH_PREDICTION_INSTANCES_FILE
)

! gsutil cp test.csv $BATCH_PREDICTION_GCS_SOURCE

### 发送批量预测请求

再次使用`batch_predict()`方法发送批量预测请求，但是将参数`instances_format`设置为"csv"。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

# The name of the job
BATCH_PREDICTION_JOB_NAME = "churn_batch-" + UUID

# Folder in the bucket to write results to
DESTINATION_FOLDER = "batch_prediction_results2"

# The Cloud Storage bucket to upload results to
BATCH_PREDICTION_GCS_DEST_PREFIX = BUCKET_URI + "/" + DESTINATION_FOLDER

# Make SDK batch_predict method call
batch_prediction_job = model.batch_predict(
    instances_format="csv",
    predictions_format="jsonl",
    job_display_name=BATCH_PREDICTION_JOB_NAME,
    gcs_source=BATCH_PREDICTION_GCS_SOURCE,
    gcs_destination_prefix=BATCH_PREDICTION_GCS_DEST_PREFIX,
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=True,
)

### 获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将被写入您在批量预测请求中指定的Cloud Storage输出存储桶。调用`iter_outputs()`方法可获取生成的结果文件列表。每个文件以JSON格式包含一个或多个预测请求：

- `instance`：预测请求。
- `prediction`：预测响应。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_prediction_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

#### 删除批量预测任务

您可以使用`delete()`方法删除您的批量预测任务。

In [ ]:
batch_prediction_job.delete()

## 使用BigQuery输入格式进行批量预测

接下来，您将执行相同的批处理作业，只是输入格式是一个BigQuery表。当输入格式是BigQuery表时，输出格式也必须是一个BigQuery表。要将BigQuery作为输入格式，您的模型必须将其输入作为值列表（数组）。

### 从CSV文件创建一个BigQuery数据集

您可以使用BigQuery的`create_dataset()`和`load_table_from_uri()`方法从CSV文件创建一个BigQuery数据集，步骤如下：

- `create_dataset()`: 创建一个空的BigQuery数据集，具有以下参数:
 - `dataset_ref`: 从数据集ID创建的`DatasetReference`，例如samples。
- `load_table_from_uri()`: 将一个或多个CSV文件加载到相应数据集中的表中，具有以下参数：
 - `url`: 一个或多个在Cloud Storage存储中的CSV文件集合。
 - `table`: 表的`TableReference`。
 - `job_config`: 如何加载CSV数据的规范。

了解更多关于[将CSV数据导入BigQuery](https://www.tensorflow.org/io/tutorials/bigquery#import_census_data_into_bigquery)。

In [ ]:
LOCATION = "us"

CSV_SCHEMA = [
    bigquery.SchemaField("sepal_length", "FLOAT"),
    bigquery.SchemaField("sepal_width", "FLOAT"),
    bigquery.SchemaField("petal_length", "FLOAT"),
    bigquery.SchemaField("petal_width", "FLOAT"),
]

DATASET_ID = "batch"
TABLE_ID = "slice1"


def create_bigquery_dataset(dataset_id):
    dataset = bigquery.Dataset(
        bigquery.dataset.DatasetReference(PROJECT_ID, dataset_id)
    )
    dataset.location = "us"

    try:
        dataset = bqclient.create_dataset(dataset)  # API request
        return True
    except Exception as err:
        print(err)
        if err.code != 409:  # http_client.CONFLICT
            raise
    return False


def load_data_into_bigquery(url, dataset_id, table_id):
    create_bigquery_dataset(dataset_id)
    dataset = bqclient.dataset(dataset_id)
    table = dataset.table(table_id)

    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.schema = CSV_SCHEMA
    job_config.skip_leading_rows = 1  # heading

    load_job = bqclient.load_table_from_uri(url, table, job_config=job_config)
    print("Starting job {}".format(load_job.job_id))

    load_job.result()  # Waits for table load to complete.
    print("Job finished.")

    destination_table = bqclient.get_table(table)
    print("Loaded {} rows.".format(destination_table.num_rows))

    return destination_table


bq_table = load_data_into_bigquery(data_file, DATASET_ID, TABLE_ID)
print(bq_table)

发送批量预测请求

再次使用`batch_predict()`方法进行批量预测请求，但参数设置如下所示:

- `instances_format`：设置为'bigquery'
- `predictions_format`：设置为'bigquery'
- 使用`bigquery_source`代替`gcs_source`。
- 使用`bigquery_destination_prefix`代替`gcs_destination_prefix`。

In [ ]:
MIN_NODES = 1
MAX_NODES = 1

# The name of the job
BATCH_PREDICTION_JOB_NAME = "churn_batch-" + UUID

# Folder in the bucket to write results to
DESTINATION_FOLDER = "batch_prediction_results_bq"

# The Cloud Storage bucket to upload results to
BATCH_PREDICTION_GCS_DEST_PREFIX = BUCKET_URI + "/" + DESTINATION_FOLDER

BQ_TABLE_SOURCE = f"bq://{PROJECT_ID}.{bq_table.dataset_id}.{bq_table.table_id}"

# Make SDK batch_predict method call
batch_prediction_job = model.batch_predict(
    instances_format="bigquery",
    predictions_format="bigquery",
    job_display_name=BATCH_PREDICTION_JOB_NAME,
    bigquery_source=BQ_TABLE_SOURCE,
    bigquery_destination_prefix=f"bq://{PROJECT_ID}.batch",
    model_parameters=None,
    machine_type=DEPLOY_COMPUTE,
    accelerator_type=DEPLOY_GPU,
    accelerator_count=DEPLOY_NGPU,
    starting_replica_count=MIN_NODES,
    max_replica_count=MAX_NODES,
    sync=True,
)

### 获取预测结果

接下来，从已完成的批处理预测作业中获取结果。

结果将写入到您指定为目标的BigQuery数据集路径上的一个BigQuery表中。批处理服务器会创建这个表，表位置由以下参数指定：

`batch_prediction_job.output_info.bigquery_output_dataset`：项目和数据集组件。
`batch_prediction_job.output_info.bigquery_output_table`：表组件。

In [ ]:
BQ_RESULTS_TABLE = (
    batch_prediction_job.output_info.bigquery_output_dataset
    + "."
    + batch_prediction_job.output_info.bigquery_output_table
)

print(BQ_RESULTS_TABLE)

table = bigquery.TableReference.from_string(BQ_RESULTS_TABLE[5:])

rows = bqclient.list_rows(table, max_results=10)

for row in rows:
    print(row)
    for key, value in row.items():
        pass

#### 删除批量预测任务

您可以使用`delete（）`方法删除您的批量预测任务。

In [ ]:
batch_prediction_job.delete()

#### 删除模型

您可以使用 `delete()` 方法删除您的模型。

In [ ]:
model.delete()

## 清理工作

要清理此项目中使用的所有Google Cloud资源，您可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源。

In [ ]:
delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -rf {BUCKET_URI}